# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('zillow_data.csv')

In [3]:
df.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
8,91940,77449,Katy,TX,Houston,Harris,9,95400.0,95600.0,95800.0,...,166800,167400,168400,169600,170900,172300,173300,174200,175400,176200
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500


# Step 2: Data Preprocessing

In [4]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [5]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [13]:
# James Irving suggestion:

# Note:  need to rename "time" column to month.  

# def make_datetimeindex(df, col_to_make_index = 'Month', drop=False, verbose=True):
#     col_to_make_index = 'Month'
#     df[col_to_make_index] = pd.to_datetime(df[col_to_make_index], errors='coerce')
#     if verbose:
#         display(df.index)
#     return 

# "coerce" makes it so that if there is a cell with something other than
# a date, it will not throw an error once run

# drop=False keeps the column with the date, as well as setting the date 
# as the index

# make_datetimeindex(df)

Note:  I have surmised that RegionName is the ZIP code for each entry.  RegionName values with only 4 digits represent ZIP codes that actually begin with '0'.  So that I can work with ZIP codes in the data set, I will need to add that zero onto every 4-digit RegionName value.  Once that's completed, I'll rename this column "ZipCode".  

In [8]:
# df.RegionName.sort_values().head(10)

df.sort_values(by="RegionName").head(20)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5850,58196,1001,Agawam,MA,Springfield,Hampden,5851,113100.0,112800.0,112600.0,...,213900,215700,218200,220100,221100,221700,221700,221700,222700,223600
4199,58197,1002,Amherst,MA,Springfield,Hampshire,4200,161000.0,160100.0,159300.0,...,333700,334800,336700,338900,340400,342000,344500,347400,350600,353300
11213,58200,1005,Barre,MA,Worcester,Worcester,11214,103100.0,103400.0,103600.0,...,205600,206800,208800,210400,211300,213300,215600,217900,219500,220700
6850,58201,1007,Belchertown,MA,Springfield,Hampshire,6851,133400.0,132700.0,132000.0,...,266100,266300,267000,267500,268000,268100,268100,268800,270000,270600
14547,58202,1008,Blandford,MA,Springfield,Hampden,14548,117500.0,117300.0,117100.0,...,202400,202900,205900,208500,207500,205400,204500,206800,210900,214200
11702,58204,1010,Brimfield,MA,Springfield,Hampden,11703,115800.0,115700.0,115500.0,...,246400,247500,249100,248800,245400,243800,244600,246700,248800,250900
14475,58205,1011,Chester,MA,Springfield,Hampden,14476,87000.0,87000.0,87100.0,...,160200,156700,156600,157700,155400,151400,150100,150100,149500,149100
4568,58207,1013,Chicopee,MA,Springfield,Hampden,4569,88500.0,88500.0,88400.0,...,167700,169300,171100,171900,172300,173300,174600,175900,176900,177400
2694,58209,1020,Chicopee,MA,Springfield,Hampden,2695,97200.0,97100.0,97000.0,...,178600,181000,184100,186100,186800,187400,188100,189000,189600,189800
14594,58212,1026,Cummington,MA,Springfield,Hampshire,14595,91200.0,91100.0,91000.0,...,193400,195900,200800,202600,201800,201200,204600,211200,217900,220800


In [9]:
df['RegionName'] = df.RegionName.astype(str)
df['RegionName'].dtype


dtype('O')

In [10]:
df.rename(columns={'RegionName': 'Zip'}, inplace=True)
df.columns
df.head()

,RegionID,Zip,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [11]:
zips = []

for i in df['Zip']:
    if len(i) < 5:
        i = '0' + i
        zips.append(i)
    else:
        zips.append(i)

zips
df['Zip'] = pd.Series(zips)
df.sort_values(by='Zip').head(10)


,RegionID,Zip,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5850,58196,01001,Agawam,MA,Springfield,Hampden,5851,113100.0,112800.0,112600.0,...,213900,215700,218200,220100,221100,221700,221700,221700,222700,223600
4199,58197,01002,Amherst,MA,Springfield,Hampshire,4200,161000.0,160100.0,159300.0,...,333700,334800,336700,338900,340400,342000,344500,347400,350600,353300
11213,58200,01005,Barre,MA,Worcester,Worcester,11214,103100.0,103400.0,103600.0,...,205600,206800,208800,210400,211300,213300,215600,217900,219500,220700
6850,58201,01007,Belchertown,MA,Springfield,Hampshire,6851,133400.0,132700.0,132000.0,...,266100,266300,267000,267500,268000,268100,268100,268800,270000,270600
14547,58202,01008,Blandford,MA,Springfield,Hampden,14548,117500.0,117300.0,117100.0,...,202400,202900,205900,208500,207500,205400,204500,206800,210900,214200
11702,58204,01010,Brimfield,MA,Springfield,Hampden,11703,115800.0,115700.0,115500.0,...,246400,247500,249100,248800,245400,243800,244600,246700,248800,250900
14475,58205,01011,Chester,MA,Springfield,Hampden,14476,87000.0,87000.0,87100.0,...,160200,156700,156600,157700,155400,151400,150100,150100,149500,149100
4568,58207,01013,Chicopee,MA,Springfield,Hampden,4569,88500.0,88500.0,88400.0,...,167700,169300,171100,171900,172300,173300,174600,175900,176900,177400
2694,58209,01020,Chicopee,MA,Springfield,Hampden,2695,97200.0,97100.0,97000.0,...,178600,181000,184100,186100,186800,187400,188100,189000,189600,189800
14594,58212,01026,Cummington,MA,Springfield,Hampshire,14595,91200.0,91100.0,91000.0,...,193400,195900,200800,202600,201800,201200,204600,211200,217900,220800


In [10]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'Zip', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted   # removed remaining part of this code:  .groupby('time').aggregate({'value':'mean'})

In [11]:
df_melt = melt_data(df)
df_melt.head()

,RegionID,Zip,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,1996-04-01,235700.0
2,91982,77494,Katy,TX,Houston,Harris,3,1996-04-01,210400.0
3,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
4,93144,79936,El Paso,TX,El Paso,El Paso,5,1996-04-01,77300.0


In [12]:
df_melt.groupby(['Zip', 'time', 'State', 'City']).aggregate({'value':'mean'}).head(40)

value
Zip   time       State City            
01001 1996-04-01 MA    Agawam  113100.0
      1996-05-01 MA    Agawam  112800.0
      1996-06-01 MA    Agawam  112600.0
      1996-07-01 MA    Agawam  112300.0
      1996-08-01 MA    Agawam  112100.0
      1996-09-01 MA    Agawam  111900.0
      1996-10-01 MA    Agawam  111800.0
      1996-11-01 MA    Agawam  111700.0
      1996-12-01 MA    Agawam  111700.0
      1997-01-01 MA    Agawam  111900.0
      1997-02-01 MA    Agawam  112100.0
      1997-03-01 MA    Agawam  112300.0
      1997-04-01 MA    Agawam  112500.0
      1997-05-01 MA    Agawam  112800.0
      1997-06-01 MA    Agawam  113000.0
      1997-07-01 MA    Agawam  113300.0
      1997-08-01 MA    Agawam  113600.0
      1997-09-01 MA    Agawam  113900.0
      1997-10-01 MA    Agawam  114200.0
      1997-11-01 MA    Agawam  114500.0
      1997-12-01 MA    Agawam  114800.0
      1998-01-01 MA    Agawam  115200.0
      1998-02-01 MA    Agawam  115600.0
      1998-03-01 MA    Agawam  115900.0
      1998-04-01 MA    Agawam  116200.0
      1998-05-01 MA    Agawam  116400.0
      1998-06-01 MA    Agawam  116600.0
      1998-07-01 MA    Agawam  116700.0
      1998-08-01 MA    Agawam  116800.0
      1998-09-01 MA    Agawam  116900.0
      1998-10-01 MA    Agawam  117100.0
      1998-11-01 MA    Agawam  117300.0
      1998-12-01 MA    Agawam  117500.0
      1999-01-01 MA    Agawam  117800.0
      1999-02-01 MA    Agawam  118300.0
      1999-03-01 MA    Agawam  118700.0
      1999-04-01 MA    Agawam  119100.0
      1999-05-01 MA    Agawam  119500.0
      1999-06-01 MA    Agawam  120000.0
      1999-07-01 MA    Agawam  120400.0

In [13]:
df_melt.groupby(['Zip', 'time', 'State', 'City']).sum().head(40)

RegionID  SizeRank     value
Zip   time       State City                                
01001 1996-04-01 MA    Agawam     58196      5851  113100.0
      1996-05-01 MA    Agawam     58196      5851  112800.0
      1996-06-01 MA    Agawam     58196      5851  112600.0
      1996-07-01 MA    Agawam     58196      5851  112300.0
      1996-08-01 MA    Agawam     58196      5851  112100.0
      1996-09-01 MA    Agawam     58196      5851  111900.0
      1996-10-01 MA    Agawam     58196      5851  111800.0
      1996-11-01 MA    Agawam     58196      5851  111700.0
      1996-12-01 MA    Agawam     58196      5851  111700.0
      1997-01-01 MA    Agawam     58196      5851  111900.0
      1997-02-01 MA    Agawam     58196      5851  112100.0
      1997-03-01 MA    Agawam     58196      5851  112300.0
      1997-04-01 MA    Agawam     58196      5851  112500.0
      1997-05-01 MA    Agawam     58196      5851  112800.0
      1997-06-01 MA    Agawam     58196      5851  113000.0
      1997-07-01 MA    Agawam     58196      5851  113300.0
      1997-08-01 MA    Agawam     58196      5851  113600.0
      1997-09-01 MA    Agawam     58196      5851  113900.0
      1997-10-01 MA    Agawam     58196      5851  114200.0
      1997-11-01 MA    Agawam     58196      5851  114500.0
      1997-12-01 MA    Agawam     58196      5851  114800.0
      1998-01-01 MA    Agawam     58196      5851  115200.0
      1998-02-01 MA    Agawam     58196      5851  115600.0
      1998-03-01 MA    Agawam     58196      5851  115900.0
      1998-04-01 MA    Agawam     58196      5851  116200.0
      1998-05-01 MA    Agawam     58196      5851  116400.0
      1998-06-01 MA    Agawam     58196      5851  116600.0
      1998-07-01 MA    Agawam     58196      5851  116700.0
      1998-08-01 MA    Agawam     58196      5851  116800.0
      1998-09-01 MA    Agawam     58196      5851  116900.0
      1998-10-01 MA    Agawam     58196      5851  117100.0
      1998-11-01 MA    Agawam     58196      5851  117300.0
      1998-12-01 MA    Agawam     58196      5851  117500.0
      1999-01-01 MA    Agawam     58196      5851  117800.0
      1999-02-01 MA    Agawam     58196      5851  118300.0
      1999-03-01 MA    Agawam     58196      5851  118700.0
      1999-04-01 MA    Agawam     58196      5851  119100.0
      1999-05-01 MA    Agawam     58196      5851  119500.0
      1999-06-01 MA    Agawam     58196      5851  120000.0
      1999-07-01 MA    Agawam     58196      5851  120400.0

In [14]:
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'}).head()

value
State City      time                     
AK    Anchorage 1996-04-01  139415.384615
                1996-05-01  139753.846154
                1996-06-01  140061.538462
                1996-07-01  140323.076923
                1996-08-01  140600.000000

In [15]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3744704 entries, 0 to 3901594
Data columns (total 9 columns):
RegionID      int64
Zip           object
City          object
State         object
Metro         object
CountyName    object
SizeRank      int64
time          datetime64[ns]
value         float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 285.7+ MB


In [16]:
df_melt.set_index('time', inplace=True)

# Step 4: Reshape from Wide to Long Format

In [17]:
df_melt.head(10)

,RegionID,Zip,City,State,Metro,CountyName,SizeRank,value
time,,,,,,,,
1996-04-01,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0
1996-04-01,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0
1996-04-01,91982,77494,Katy,TX,Houston,Harris,3,210400.0
1996-04-01,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0
1996-04-01,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0
1996-04-01,91733,77084,Houston,TX,Houston,Harris,6,95000.0
1996-04-01,61807,10467,New York,NY,New York,Bronx,7,152900.0
1996-04-01,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0
1996-04-01,91940,77449,Katy,TX,Houston,Harris,9,95400.0


In [18]:
df_melt.loc[df_melt['State'] == 'AK'].groupby(['time']).aggregate({'value':'mean'}).head()

,value
time,
1996-04-01,128382.142857
1996-05-01,128832.142857
1996-06-01,129253.571429
1996-07-01,129639.285714
1996-08-01,130028.571429


In [19]:
df_melt['1997':].head(10)
df_melt.loc[df_melt['State'] == 'MA'].groupby(['RegionID', 'Zip', 'time']).mean().head(10)

SizeRank     value
RegionID Zip   time                          
58196    01001 1996-04-01      5851  113100.0
               1996-05-01      5851  112800.0
               1996-06-01      5851  112600.0
               1996-07-01      5851  112300.0
               1996-08-01      5851  112100.0
               1996-09-01      5851  111900.0
               1996-10-01      5851  111800.0
               1996-11-01      5851  111700.0
               1996-12-01      5851  111700.0
               1997-01-01      5851  111900.0

In [20]:
# df_melt_annual = df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
# df_melt_annual.loc[df_melt_annual['City'] =='Chicago'].aggregate({'value':'mean'})
                                # .groupby(['time']).aggregate({'value':'mean'}).head()
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
df_melt.loc[df_melt['City']=='Chicago'].head(10)

,RegionID,Zip,City,State,Metro,CountyName,SizeRank,value
time,,,,,,,,
1996-04-01,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0
1996-04-01,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0
1996-04-01,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0
1996-04-01,84646,60647,Chicago,IL,Chicago,Cook,17,122700.0
1996-04-01,84620,60618,Chicago,IL,Chicago,Cook,20,142600.0
1996-04-01,84615,60613,Chicago,IL,Chicago,Cook,41,297900.0
1996-04-01,84630,60629,Chicago,IL,Chicago,Cook,49,93400.0
1996-04-01,84627,60625,Chicago,IL,Chicago,Cook,51,148900.0
1996-04-01,84621,60619,Chicago,IL,Chicago,Cook,75,76400.0


In [ ]:
df_melt.drop(['RegionID', 'SizeRank'], axis = 1, inplace=True)


In [28]:
df_melt.head()

,Zip,City,State,Metro,CountyName,value
time,,,,,,
1996-04-01,60657,Chicago,IL,Chicago,Cook,334200.0
1996-04-01,75070,McKinney,TX,Dallas-Fort Worth,Collin,235700.0
1996-04-01,77494,Katy,TX,Houston,Harris,210400.0
1996-04-01,60614,Chicago,IL,Chicago,Cook,498100.0
1996-04-01,79936,El Paso,TX,El Paso,El Paso,77300.0


In [30]:
df_annual = df_melt

In [31]:
df_annual.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3744704 entries, 1996-04-01 to 2018-04-01
Data columns (total 6 columns):
Zip           object
City          object
State         object
Metro         object
CountyName    object
value         float64
dtypes: float64(1), object(5)
memory usage: 280.0+ MB


In [34]:
df_annual.groupby(['Zip', 'City', 'State', 'Metro', 'CountyName']).resample('A').mean().head(20)

value
Zip   City   State Metro       CountyName time                     
01001 Agawam MA    Springfield Hampden    1996-12-31  112222.222222
                                          1997-12-31  113241.666667
                                          1998-12-31  116516.666667
                                          1999-12-31  120200.000000
                                          2000-12-31  126225.000000
                                          2001-12-31  134366.666667
                                          2002-12-31  146408.333333
                                          2003-12-31  161691.666667
                                          2004-12-31  178733.333333
                                          2005-12-31  206633.333333
                                          2006-12-31  221558.333333
                                          2007-12-31  219683.333333
                                          2008-12-31  207125.000000
                                          2009-12-31  198833.333333
                                          2010-12-31  196550.000000
                                          2011-12-31  188508.333333
                                          2012-12-31  183250.000000
                                          2013-12-31  186100.000000
                                          2014-12-31  186008.333333
                                          2015-12-31  191916.666667

In [35]:
df_annual['value_rounded'] = round(df_annual['value'], 2)
df_annual.head()

,Zip,City,State,Metro,CountyName,value,value_rounded
time,,,,,,,
1996-04-01,60657,Chicago,IL,Chicago,Cook,334200.0,334200.0
1996-04-01,75070,McKinney,TX,Dallas-Fort Worth,Collin,235700.0,235700.0
1996-04-01,77494,Katy,TX,Houston,Harris,210400.0,210400.0
1996-04-01,60614,Chicago,IL,Chicago,Cook,498100.0,498100.0
1996-04-01,79936,El Paso,TX,El Paso,El Paso,77300.0,77300.0


In [36]:
df_annual.columns

Index(['Zip', 'City', 'State', 'Metro', 'CountyName', 'value',
       'value_rounded'],
      dtype='object')

In [40]:
df_annual.sort_values(by=['Zip', 'time']).head(20)

,Zip,City,State,Metro,CountyName,value,value_rounded
time,,,,,,,
1996-04-01,01001,Agawam,MA,Springfield,Hampden,113100.0,113100.0
1996-05-01,01001,Agawam,MA,Springfield,Hampden,112800.0,112800.0
1996-06-01,01001,Agawam,MA,Springfield,Hampden,112600.0,112600.0
1996-07-01,01001,Agawam,MA,Springfield,Hampden,112300.0,112300.0
1996-08-01,01001,Agawam,MA,Springfield,Hampden,112100.0,112100.0
1996-09-01,01001,Agawam,MA,Springfield,Hampden,111900.0,111900.0
1996-10-01,01001,Agawam,MA,Springfield,Hampden,111800.0,111800.0
1996-11-01,01001,Agawam,MA,Springfield,Hampden,111700.0,111700.0
1996-12-01,01001,Agawam,MA,Springfield,Hampden,111700.0,111700.0


In [45]:
df_annual.sort_values(by=['Zip', 'time', 'City', 'State', 'Metro']).resample('A').mean().head(20)

,value,value_rounded
time,,
1996-12-31,118863.044431,118863.044431
1997-12-31,122365.248395,122365.248395
1998-12-31,129392.784516,129392.784516
1999-12-31,138962.489345,138962.489345
2000-12-31,151834.752563,151834.752563
2001-12-31,165125.207618,165125.207618
2002-12-31,178814.105234,178814.105234
2003-12-31,195592.192880,195592.192880
2004-12-31,219909.882762,219909.882762


In [54]:
df_annual[df_annual['State'] == 'CA'].groupby(['State', 'Metro', 'City','Zip']).resample('A').mean().head(60)      #sort_values(by='time')

value  value_rounded
State Metro       City        Zip   time                                    
CA    Bakersfield Arvin       93203 1996-12-31   72311.111111   72311.111111
                                    1997-12-31   72341.666667   72341.666667
                                    1998-12-31   74775.000000   74775.000000
                                    1999-12-31   74491.666667   74491.666667
                                    2000-12-31   74158.333333   74158.333333
                                    2001-12-31   80275.000000   80275.000000
                                    2002-12-31   87450.000000   87450.000000
                                    2003-12-31   95991.666667   95991.666667
                                    2004-12-31  111541.666667  111541.666667
                                    2005-12-31  157866.666667  157866.666667
                                    2006-12-31  199016.666667  199016.666667
                                    2007-12-31  199275.000000  199275.000000
                                    2008-12-31  160866.666667  160866.666667
                                    2009-12-31  121750.000000  121750.000000
                                    2010-12-31   97283.333333   97283.333333
                                    2011-12-31   88825.000000   88825.000000
                                    2012-12-31   86233.333333   86233.333333
                                    2013-12-31  102908.333333  102908.333333
                                    2014-12-31  125508.333333  125508.333333
                                    2015-12-31  144608.333333  144608.333333
                                    2016-12-31  151358.333333  151358.333333
                                    2017-12-31  154041.666667  154041.666667
                                    2018-12-31  159925.000000  159925.000000
                  Bakersfield 93301 2014-12-31  163875.000000  163875.000000
                                    2015-12-31  178641.666667  178641.666667
                                    2016-12-31  181600.000000  181600.000000
                                    2017-12-31  185166.666667  185166.666667
                                    2018-12-31  194625.000000  194625.000000
                              93304 1996-12-31   70655.555556   70655.555556
                                    1997-12-31   68683.333333   68683.333333
                                    1998-12-31   70100.000000   70100.000000
                                    1999-12-31   73116.666667   73116.666667
                                    2000-12-31   74958.333333   74958.333333
                                    2001-12-31   77791.666667   77791.666667
                                    2002-12-31   85925.000000   85925.000000
                                    2003-12-31  100708.333333  100708.333333
                                    2004-12-31  136408.333333  136408.333333
                                    2005-12-31  199483.333333  199483.333333
                                    2006-12-31  235508.333333  235508.333333
                                    2007-12-31  214616.666667  214616.666667
                                    2008-12-31  148441.666667  148441.666667
                                    2009-12-31   89116.666667   89116.666667
                                    2010-12-31   79075.000000   79075.000000
                                    2011-12-31   78441.666667   78441.666667
                                    2012-12-31   78408.333333   78408.333333
                                    2013-12-31  102800.000000  102800.000000
                                    2014-12-31  123175.000000  123175.000000
                                    2015-12-31  134358.333333  134358.333333
                                    2016-12-31  142075.000000  142075.000000
                                    2017-12-31  149900.000000  149900.000000
                                    2018-12-31  1556

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

matplotlib.rc('font', **font)

df.plot(figsize=(12, 6));

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [52]:
df_annual[df_annual['State'] == 'SD'].groupby(['Zip', 'City', 'Metro', 'State']).resample('A').mean()    #sort_values(by='time')

value  value_rounded
Zip   City    Metro       State time                                    
57003 Baltic  Sioux Falls SD    1996-12-31   38766.666667   38766.666667
                                1997-12-31   47125.000000   47125.000000
                                1998-12-31   61666.666667   61666.666667
                                1999-12-31   80383.333333   80383.333333
                                2000-12-31   94916.666667   94916.666667
                                2001-12-31  103183.333333  103183.333333
                                2002-12-31  109291.666667  109291.666667
                                2003-12-31  117758.333333  117758.333333
                                2004-12-31  133216.666667  133216.666667
                                2005-12-31  138083.333333  138083.333333
                                2006-12-31  141691.666667  141691.666667
                                2007-12-31  142850.000000  142850.000000
                                2008-12-31  142250.000000  142250.000000
                                2009-12-31  144308.333333  144308.333333
                                2010-12-31  137675.000000  137675.000000
                                2011-12-31  133883.333333  133883.333333
                                2012-12-31  136091.666667  136091.666667
                                2013-12-31  141800.000000  141800.000000
                                2014-12-31  149550.000000  149550.000000
                                2015-12-31  156633.333333  156633.333333
                                2016-12-31  165941.666667  165941.666667
                                2017-12-31  173508.333333  173508.333333
                                2018-12-31  182275.000000  182275.000000
57005 Brandon Sioux Falls SD    1996-12-31   36577.777778   36577.777778
                                1997-12-31   45658.333333   45658.333333
                                1998-12-31   62425.000000   62425.000000
                                1999-12-31   91575.000000   91575.000000
                                2000-12-31  122591.666667  122591.666667
                                2001-12-31  132650.000000  132650.000000
                                2002-12-31  135591.666667  135591.666667
                                2003-12-31  141125.000000  141125.000000
                                2004-12-31  150691.666667  150691.666667
                                2005-12-31  153900.000000  153900.000000
                                2006-12-31  160641.666667  160641.666667
                                2007-12-31  163000.000000  163000.000000
                                2008-12-31  164516.666667  164516.666667
                                2009-12-31  163058.333333  163058.333333
                                2010-12-31  157316.666667  157316.666667
                                2011-12-31  158166.666667  158166.666667
                                2012-12-31  168250.000000  168250.000000

In [12]:
from fsds_100719.imports import *

## Project Notebook Settings
pd.set_option('display.max_columns',0)

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-notebook')

fsds_1007219  v0.6.6 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [2]:
!pip install -U fsds_100719


Requirement not upgraded as not directly required: numpy in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from fsds_100719) (1.15.4)
Requirement not upgraded as not directly required: lxml in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from fsds_100719) (4.3.3)
    100% |████████████████████████████████| 122kB 1.9MB/s ta 0:00:01
Requirement not upgraded as not directly required: IPython in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from fsds_100719) (6.5.0)
Requirement not upgraded as not directly required: seaborn in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from fsds_100719) (0.9.0)
  Using cached https://files.pythonhosted.org/packages/ce/e7/37c4bc1416d01102d792dac3cb1ebe4b62d5e5e1e585dbfb3e02d8ebd484/wordcloud-1.6.0-cp36-cp36m-macosx_10_6_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/80/d6/4294f0b4bce4de0abf13e17190289f9d0613b0a44e5dd6a7f5ca98459853/selenium-3.141.0-py2.py3-

Requirement not upgraded as not directly required: notebook>=4.4.1 in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from widgetsnbextension~=3.5.0->ipywidgets->fsds_100719) (5.7.4)
Requirement not upgraded as not directly required: ipython_genutils in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from traitlets>=4.3.1->ipywidgets->fsds_100719) (0.2.0)
Requirement not upgraded as not directly required: six in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from traitlets>=4.3.1->ipywidgets->fsds_100719) (1.11.0)
Requirement not upgraded as not directly required: jsonschema!=2.5.0,>=2.4 in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from nbformat>=4.2.0->ipywidgets->fsds_100719) (2.6.0)
Requirement not upgraded as not directly required: jupyter_core in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from nbformat>=4.2.0->ipywidgets->fsds_100719) (4.4.0)
Requirement not upgraded as not direct

Requirement not upgraded as not directly required: webencodings in /Users/Gina/anaconda3/envs/learn-env/lib/python3.6/site-packages (from bleach->nbconvert>=5.3.1->phik>=0.9.8->pandas-profiling->fsds_100719) (0.5.1)
    100% |████████████████████████████████| 225kB 24.4MB/s ta 0:00:01
  Installing build dependencies ... done
  Running setup.py bdist_wheel for pyperclip ... done
  Stored in directory: /Users/Gina/Library/Caches/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
  Running setup.py bdist_wheel for pandas-profiling ... done
  Stored in directory: /Users/Gina/Library/Caches/pip/wheels/b2/4b/40/ff4633a59f4e7fbdfec19b8e10e353b54b5eef125c31afe51a
  Running setup.py bdist_wheel for confuse ... done
  Stored in directory: /Users/Gina/Library/Caches/pip/wheels/b0/b2/96/2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
  Running setup.py bdist_wheel for htmlmin ... done
  Stored in directory: /Users/Gina/Library/Caches/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results